STEP 1) Load and import modules.

In [ ]:
import numpy as np
import sep

from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [ ]:
%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

STEP 2) Read in data from .fits file

In [ ]:
data = fits.getdata("./Final Project/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
f125w = fits.getdata("./Final Project/hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
f160w = fits.getdata("./Final Project/hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits")

data = data.byteswap().newbyteorder()
f125w = f125w.byteswap().newbyteorder()
f160w = f160w.byteswap().newbyteorder()

Skip to END OF PART 8 to look at the other parts first and the contineuation of STEP 3.

----------------------------- SART OF PART 8 -----------------------------

Removing the background.

In [ ]:
bkg_f105w = sep.Background(data)
bkg_f125w = sep.Background(f125w)
bkg_f160w = sep.Background(f160w)

f105w = data - bkg_f105w
f125w -= bkg_f125w
f160w -= bkg_f160w

Defining the rescale data function to rescale the data to our purposes.

In [ ]:
def rescale_data(d, dmin=0.01, dmax=1.0):
    
    datac = np.clip(d, dmin, dmax)
    return (np.log10(datac) - np.log10(dmin)) / (np.log10(dmax) - np.log10(dmin))

Getting rescaled versions of the data.

In [ ]:
p_f105w = rescale_data(f105w)
p_f125w = rescale_data(f125w)
p_f160w = rescale_data(f160w)

Getting RGB stack.

In [ ]:
rgb = np.stack([p_f160w, 0.7 * p_f125w, p_f105w], axis = -1)
print(rgb.shape)

Showing the data.

In [ ]:
xsize = rgb.shape[1]
ysize = rgb.shape[0]

f, ax = plt.subplots(1, 1, figsize=(xsize/1000., ysize/1000.0))
ax.axis('off')
ax.imshow(rgb, origin = 'lower')

In [ ]:
rgb = np.clip(rgb, 0, 1) # Error keeps poping up where float has to be within 0 and 1
plt.imsave('3-color_false_image.png', rgb, origin='lower')

----------------------------- END OF PART 8 -----------------------------

Showing the image.

In [ ]:
m, s = np.nanmedian(data), np.std(data)

plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

STEP 3) Remove the background.

Measure a spatially varying background on the image. (getting the background of the image)

In [ ]:
bkg = sep.Background(data)

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
bkg_image = bkg.back()

Show the backround.

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("remove_background_part1.png")

In [ ]:
bkg_rms = bkg.rms()

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig("remove_background_part2.png")

Remove the sky background from the image and displaying the image.

In [ ]:
data_sub = data - bkg

STEP 4) Detecting objects

Setting the detection threashhold and finding the array of objects.

In [ ]:
objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)

In [ ]:
len(objects)

Importing ellipse from matplotlib.

In [ ]:
from matplotlib.patches import Ellipse

Plotting the image

In [ ]:
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig("object_detection.png")

STEP 5)Object and fluxes analysis

In [ ]:
objects.dtype.names

In [ ]:
print(f"There are {len(objects)} objects.")

Creating a simple circular aperture photometry at the locations of the objects.(Basically circling the objects)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

Creating and displaying a histogram of the objects.

In [ ]:
mag = -2.5*np.log10(flux)

plt.hist(mag, range=[-5,10], bins=1000, alpha=0.5)
plt.xlabel('-2.5 log10 F444W')
plt.ylabel('N')

plt.savefig('flux_histogram.png')

Finding the standard mean, median, and standard deviation of the fluxes.

In [ ]:
f_mean = np.mean(flux)
f_median = np.nanmedian(flux)
f_std = np.std(flux)

print(f"The mean of the fluxes is {f_mean}")
print(f"The median of the fluxes is {f_median}")
print(f"The standard deviation of the fluxes is {f_std}")

In [ ]:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

Finding the outliers using the IQR.

In [ ]:
q1 = np.percentile(flux, 25)
q3 = np.percentile(flux, 75)
iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

f_outliers = flux[(flux < lower_bound) | (flux > upper_bound)]

# Largest outlier
print(f"The largest outlier of the flux is {max(f_outliers)}.")